In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests and Decision Trees, but now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 Lending data.

In [2]:
y2015 = pd.read_csv('/Users/abnussbacher/Downloads/LoanStats3d.csv', skipinitialspace=True, header=1)

NameError: name 'pd' is not defined

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use get_dummies from pandas and see what happens off of this kind of naive approach.

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN! You can restart the kernel in the 'Kernel' menu at the top of the page).

## Data Cleaning

Well, get_dummies can be a very memory intensive thing, particularly if things are typed poorly. We got a warning about that earlier. Mixed types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If theres a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all out categorical variables and see how many distinct counts there are...

In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [39]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(
                    y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line',
            'revol_util', 'sub_grade', 'addr_state', 'desc'],
            1, inplace=True)

# Remove two summary rows that don't actually contain data
y2015 = y2015[:-2]

Note that in `pd.to_numeric` we used the 'coerce' option for our conversion. This makes it so that if the function cannot figure out how to convert to numeric it returns a null for that value rather than returning an error for the whole process.

Now this should be better. Let's try again.

In [ ]:
pd.get_dummies(y2015)

It works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

In [47]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([ 0.9787229 ,  0.98031393,  0.98176248,  0.98185747,  0.97546901,
        0.97625267,  0.93331592,  0.97998053,  0.97959959,  0.98054909])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us onluy 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matricies.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [ ]:
# Your code here



In [ ]:
# EXAMPLE SOLUTION

In [51]:
df = pd.DataFrame()
df['imp']= rfc.fit(X, Y).feature_importances_
df['features'] = X.columns

In [61]:
x = X.drop(['last_pymnt_amnt', 'out_prncp', 'last_pymnt_d_Jan-2017', 'out_prncp_inv'], 1)

In [62]:
cross_val_score(rfc, x, Y, cv=10)

array([ 0.97691814,  0.97967277,  0.98062264,  0.98059889,  0.97318927,
        0.97941107,  0.90975801,  0.97781947,  0.9503408 ,  0.95658576])

In [58]:
df.sort_values(by='imp', ascending=False)

,imp,features
24,1.566652e-01,last_pymnt_amnt
15,1.409654e-01,out_prncp
152,1.286368e-01,last_pymnt_d_Jan-2017
16,9.926417e-02,out_prncp_inv
167,7.363359e-02,next_pymnt_d_Feb-2017
182,6.249018e-02,last_credit_pull_d_Jan-2017
19,4.930034e-02,total_rec_prncp
147,2.987815e-02,last_pymnt_d_Dec-2016
17,2.503673e-02,total_pymnt
18,2.434818e-02,total_pymnt_inv
